In [ ]:
import ee
import os
import geopandas as gpd
%autosave 30

In [ ]:
# # Authenticate Google Earth Engine if not done so
# ee.Authenticate()

In [ ]:
# Initialize Google Earth Engine
try:
    ee.Initialize()
except:
    os.environ['HTTP_PROXY'] = 'http://127.0.0.1:7890'
    os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'
    ee.Initialize()

In [ ]:
# Define some global functions
def getNDSI(image): # this function calculates a series of spectral indices
    ndti = image.normalizedDifference(['B11','B12']).rename('ndti').float()
    ndre = image.normalizedDifference(['B5','B4']).rename('ndre').float()
    ndvi = image.normalizedDifference(['B8','B4']).rename('ndvi').float()
    ndwi = image.normalizedDifference(['B3','B8']).rename('ndwi').float()
    mndwi = image.normalizedDifference(['B3','B11']).rename('mndwi').float()
    return image.addBands([ndti,ndre,ndvi,ndwi,mndwi])

def maskS2clouds(img): # this function perform cloud masking to Sentinel-2 images
    img = img.clip(city)
    qa=img.select('QA60')
    # Bits 10 and 11 are clouds and cirrus, respectively.
    mask1 = qa.bitwiseAnd(1 << 10).eq(0)
    mask2 = qa.bitwiseAnd(1 << 11).eq(0)
    return getNDSI(img.updateMask(mask1).updateMask(mask2))

def to_float(band): # this function converts data type of image band to float
    return img.select(band).toFloat()

In [ ]:
start = '2017-01-01'# define the first day to collect images

your_folder = 'your_folder' # define a folder on Google Drive to save the images

st = ee.ImageCollection("COPERNICUS/S2") # define the image collection to download (Sentinel-2)

# Define a water mask
jrc_water = ee.ImageCollection("JRC/GSW1_3/YearlyHistory") # define water mask to use (JRC Yearly Water Classification History, v1.3)
water_mask_all = ee.Image(jrc_water.filterDate('2016-1-1','2016-12-31').first()).unmask()

# Import the collection of cities boundaries from Google Earth Engine
# Need to upload the city boundary shapefile to Google Earth Engine fist
cities_GEE = ee.FeatureCollection("projects/salurbal-backcast/assets/Boundaries/UGS_boundaries")

# Get the list of cities from the local machine
cities = gpd.read_file(r'../Boundaries/Boundaries.shp')
IDs = cities['ID'].to_list()

In [ ]:
for ID in ['CO_001']: # use this line for toy example
# for ID in IDs: use this line for full example
    print(ID)
    # Get the city boundary and water mask in Google Earth Engine
    city = cities_GEE.filterMetadata('ID','equals',ID).first().geometry()
    region = city.bounds().getInfo()['coordinates']
    water_mask = water_mask_all.lte(1).clip(city);

    # Get images and apply cloud mask, the the median composite, and apply water mask
    imgs = st.filterBounds(city).filterDate('2017-01-01','2017-12-31').map(maskS2clouds)
    img = imgs.median().clip(city).updateMask(water_mask)
 
    # Get texture uisng Gray-Level Co-Occurence Matrix
    glcm = img.select('B8').int32().glcmTexture(size=4).select('B8_contrast').rename('glcm').float()
    img =  img.addBands(glcm)
    
    # Select the bands of interest
    bands=['ndti','ndre','ndvi','ndwi','mndwi','glcm','B2','B3','B4','B8']
    img_scale = ee.Image([to_float(band) for band in bands])
  
    # export img
    task = ee.batch.Export.image.toDrive(img_scale,
                                         folder= your_folder,description=str(ID),fileNamePrefix=str(ID),
                                         region=region,scale=10,crs='EPSG:3857'
                                         ,maxPixels=1e11,skipEmptyTiles=True)
    task.start()

In [ ]:
# check data processing status on Google Earth Engine
ee.batch.Task.list()